In [1]:
from edf.pc_utils import draw_geometry, voxel_filter
from edf.data import PointCloud, SE3, PickDemo, PlaceDemo, PickAndPlaceDemo, PickAndPlaceDataset
from edf.preprocess import Rescale, NormalizeColor, Downsample
from edf.agent import PickAgent

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
device = 'cuda:0'
# device = 'cpu'
unit_len = 0.01

In [3]:
pick_agent_config_dir = "config/agent_config/pick_agent.yaml"
max_N_query_pick = 1
langevin_dt_pick = 0.001


pick_agent = PickAgent(config_dir=pick_agent_config_dir, 
                       device = device,
                       max_N_query = max_N_query_pick, 
                       langevin_dt = langevin_dt_pick)

In [4]:
transforms = Compose([Downsample(voxel_size=unit_len * 1.0, coord_reduction="average"),
                      Rescale(rescale_factor=1/unit_len),
                      NormalizeColor(color_mean = torch.tensor([0.5, 0.5, 0.5]), color_std = torch.tensor([0.5, 0.5, 0.5])),
                      ])
trainset = PickAndPlaceDataset(dataset_dir="demo/test_demo", dataset_name="train", transforms=transforms, device=device)
train_dataloader = DataLoader(trainset, shuffle=False, collate_fn=lambda x:x)

In [5]:
demo = next(iter(train_dataloader))[0]

In [6]:
demo.pick_demo.scene_pc.points.max(dim=0).values

tensor([ 9.6347, 29.5414, 49.4110], device='cuda:0')

In [7]:
demo.pick_demo.scene_pc.points.min(dim=0).values

tensor([-49.4785, -29.5414,  34.5485], device='cuda:0')

In [ ]:
T_seed = 100
pick_policy = 'sorted'
pick_mh_iter = 1000
pick_langevin_iter = 300
pick_dist_temp = 1.
pick_policy_temp = 1.
pick_optim_iter = 100
pick_optim_lr = 0.005

In [ ]:
Ts, edf_outputs, logs = pick_agent.forward(pc=demo.pick_demo.scene_pc, T_seed=T_seed, policy = pick_policy, mh_iter=pick_mh_iter, langevin_iter=pick_langevin_iter, 
                                            temperature=pick_dist_temp, policy_temperature=pick_policy_temp, optim_iter=pick_optim_iter, optim_lr=pick_optim_lr)

In [ ]:
# draw_geometry(pc.to_pcd())
draw_geometry(train_demo.pick_demo.scene_pc.to_pcd())